<a href="https://colab.research.google.com/github/AlexeyTri/PyTorchTutorials/blob/main/Tutorial14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SAVE_LOAD**

In [1]:
import torch
import torch.nn as nn

**3 главных метода для запоминания:**

- torch.save(arg, PATH) используют для модели, тензора, словаря параметров модели

- torch.load(PATH)

- torch.load_state_dict(arg) загрузка словаря параметров

**два основных метода для save**

1) lazy way: save whole model
torch.save(model, PATH)

! model class must be diferent somewhere
model = torch.load(PATH)
model.eval() !remember that you must call model.eval() to set dropout and batch normalization layers to evaluation mode before running inference.

2) recomended way: save only the state_dict
torch.save(model.state_dict(), PATH)

! model must be created again with parameters
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

In [6]:
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [7]:
model = Model(n_input_features=6)
# train model

# save all

In [8]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.3525,  0.1465,  0.0286, -0.3435, -0.3470, -0.1081]],
       requires_grad=True)
Parameter containing:
tensor([-0.2313], requires_grad=True)


In [9]:
PATH = '/content/sample_data/model.pth'
torch.save(model, PATH)

In [10]:
loaded_model = torch.load(PATH)
loaded_model.eval()

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [11]:
for param in loaded_model.parameters():
    print(param)

Parameter containing:
tensor([[-0.3525,  0.1465,  0.0286, -0.3435, -0.3470, -0.1081]],
       requires_grad=True)
Parameter containing:
tensor([-0.2313], requires_grad=True)


save only state_dict

In [13]:
PATH = '/content/sample_data/model2.pth'
torch.save(model.state_dict(), PATH)

In [14]:
print(model.state_dict())

OrderedDict([('linear.weight', tensor([[-0.3525,  0.1465,  0.0286, -0.3435, -0.3470, -0.1081]])), ('linear.bias', tensor([-0.2313]))])


In [18]:
load_model = Model(n_input_features=6)
load_model.load_state_dict(torch.load(PATH))
print(load_model.state_dict())

OrderedDict([('linear.weight', tensor([[-0.3525,  0.1465,  0.0286, -0.3435, -0.3470, -0.1081]])), ('linear.bias', tensor([-0.2313]))])


load checkpoint

In [20]:
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [21]:
checkpoint={
    "epoch": 50,
    "model_state": model.state_dict(),
    "optim_state": optimizer.state_dict()
}

In [22]:
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'params': [0, 1]}]}


In [23]:
torch.save(checkpoint, '/content/sample_data/checkpoint.pth')

In [24]:
checkpoint = torch.load('/content/sample_data/checkpoint.pth')

In [25]:
model = Model(n_input_features=6)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [26]:
model.load_state_dict(checkpoint['model_state'])
optimizer.load_state_dict(checkpoint['optim_state'])
epoch = checkpoint['epoch']

In [27]:
model.eval() # -or- model.train()
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'params': [0, 1]}]}


# !

* remember that you must call model.eval() to set dropout and batch notmalization layers

* to evaluation mode before running inference. Failing to do thid will yeild inconsisntent inference results. If you wish to resuming training, call model.train() to ensure these layers are in training mode

""" SAVING ON GPU/CPU 

# 1) Save on GPU, Load on CPU
device = torch.device("cuda")
model.to(device)
torch.save(model.state_dict(), PATH)

device = torch.device('cpu')
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location=device))

# 2) Save on GPU, Load on GPU
device = torch.device("cuda")
model.to(device)
torch.save(model.state_dict(), PATH)

model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.to(device)

# Note: Be sure to use the .to(torch.device('cuda')) function 
# on all model inputs, too!

# 3) Save on CPU, Load on GPU
torch.save(model.state_dict(), PATH)

device = torch.device("cuda")
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location="cuda:0"))  # Choose whatever GPU device number you want
model.to(device)

# This loads the model to a given GPU device. 
# Next, be sure to call model.to(torch.device('cuda')) to convert the model’s parameter tensors to CUDA tensors
"""
